In [1]:
import os

import datetime
import requests
import time
import json

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, auc, accuracy_score, precision_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import lightgbm as lgb
import gc
from tqdm import tqdm  # 进度条


In [2]:
# 构造训练数据集
def get_all_train_data(train_path):
    all_train_data = pd.DataFrame()
    ID = 0
    print(len(os.listdir(train_path)))
    for now_csv in tqdm(os.listdir(train_path)):
        data = pd.read_csv(os.path.join(train_path, now_csv))
        all_train_data = all_train_data.append(data)
    all_train_data['end_time'] = '2020-03-10'
    all_train_data['告警开始时间'] = pd.to_datetime(all_train_data['告警开始时间'], format='%Y-%m-%d %H:%M:%S')
    all_train_data['end_time'] = pd.to_datetime(all_train_data['end_time'], format='%Y-%m-%d')
    all_train_data['time_gap'] = all_train_data.apply(lambda x: (x['end_time'] - x['告警开始时间']).days, axis=1)
    return all_train_data


# 生成的训练数据
def get_train_data(all_data, times=1):
    save_path = './all_train_data_sample_' + str(times) + '.csv'
#     if not os.path.exists(save_path):
    res_data = gener_train_data(all_data, times)
    res_data.to_csv(save_path, index=False)
    res_data = pd.read_csv(save_path)
    return res_data


# 构造特征
def gener_fea(data):
    res = data[['ID', 'end_time', 'label']].drop_duplicates()
    # 构造特征
    # 1、统计每个样本在1，2，3，4，5，6，7天内故障出现的总次数
    tmp = data.groupby(by=['ID', 'time_gap']).agg({'基站名称': 'count'}).reset_index()
    tmp = pd.pivot_table(tmp, index='ID', columns='time_gap', values='基站名称').reset_index()
    # 7天内故障的总次数
    tmp['sum_guzhang_7'] = tmp[[1, 2, 3, 4, 5, 6, 7]].apply(lambda x: x.sum(), axis=1)
    tmp.rename(columns={1: 'guzhang_1', 2: 'guzhang_2', 3: 'guzhang_3', 4: 'guzhang_4', 5: 'guzhang_5', 6: 'guzhang_6',
                        7: 'guzhang_7'}, inplace=True)
    res = res.merge(tmp, on='ID', how='left')

    # 7天内出现故障的总类型数量
    tmp = data.groupby(by=['ID']).agg({'告警名称': 'nunique'}).reset_index().rename(columns={'告警名称': '7_gaojing_nunique'})
    res = res.merge(tmp, on='ID', how='left')

    # 每天出现故障的类型数量
    tmp = data.groupby(by=['ID', 'time_gap']).agg({'告警名称': 'nunique'}).reset_index()
    tmp = pd.pivot_table(tmp, index='ID', columns='time_gap', values='告警名称').reset_index()

    tmp.rename(columns={1: 'guzhang_types_1', 2: 'guzhang_types_2', 3: 'guzhang_types_3', 4: 'guzhang_types_4',
                        5: 'guzhang_types_5',
                        6: 'guzhang_types_6', 7: 'guzhang_types_7'}, inplace=True)
    res = res.merge(tmp, on='ID', how='left')

    # 7天内出现故障的天数
    tmp = data.groupby(by=['ID']).agg({'time_gap': 'nunique'}).reset_index().rename(
        columns={'time_gap': 'time_gap_nunique'})
    res = res.merge(tmp, on='ID', how='left')
    # 7天内，平均每天的故障次数
    res['sum_guzhang_7/7'] = res['sum_guzhang_7'] / 7

    # 发生故障时，平均每天的故障次数
    res['sum_guzhang_7/time_gap_nunique'] = res['sum_guzhang_7'] / res['time_gap_nunique']

    # 故障告警、异常告警、失败告警、
    def get_guzhang(x, gaojing_type):
        res = 0
        for i in x:
            if i.find(gaojing_type) != -1:
                res += 1
        return res

    # 故障类告警的次数
    tmp = data.groupby(by=['ID'])['告警名称'].apply(
        lambda x: get_guzhang(x, '故障')).reset_index().rename(columns={'告警名称': 'guzhang_sum'})
    res = res.merge(tmp, on='ID', how='left')

    # 异常类告警的次数
    tmp = data.groupby(by=['ID'])['告警名称'].apply(
        lambda x: get_guzhang(x, '异常')).reset_index().rename(columns={'告警名称': 'yichang_sum'})
    res = res.merge(tmp, on='ID', how='left')

    # 失败类告警的次数
    tmp = data.groupby(by=['ID'])['告警名称'].apply(
        lambda x: get_guzhang(x, '失败')).reset_index().rename(columns={'告警名称': 'shibai_sum'})
    res = res.merge(tmp, on='ID', how='left')

    # 小区类告警的次数
    tmp = data.groupby(by=['ID'])['告警名称'].apply(
        lambda x: get_guzhang(x, '小区')).reset_index().rename(columns={'告警名称': 'xiaoqu_sum'})
    res = res.merge(tmp, on='ID', how='left')

    # 射频类告警的次数
    tmp = data.groupby(by=['ID'])['告警名称'].apply(
        lambda x: get_guzhang(x, '射频')).reset_index().rename(columns={'告警名称': 'shepin_sum'})
    res = res.merge(tmp, on='ID', how='left')

    # BBU类告警的次数
    tmp = data.groupby(by=['ID'])['告警名称'].apply(
        lambda x: get_guzhang(x, 'BBU')).reset_index().rename(columns={'告警名称': 'BBU_sum'})
    res = res.merge(tmp, on='ID', how='left')

    # RHUB类告警的次数
    tmp = data.groupby(by=['ID'])['告警名称'].apply(
        lambda x: get_guzhang(x, 'RHUB')).reset_index().rename(columns={'告警名称': 'RHUB_sum'})
    res = res.merge(tmp, on='ID', how='left')

    # 射频类告警的次数
    tmp = data.groupby(by=['ID'])['告警名称'].apply(
        lambda x: get_guzhang(x, 'RRU')).reset_index().rename(columns={'告警名称': 'RRU_sum'})
    res = res.merge(tmp, on='ID', how='left')

    # 7天内发生 网元连接中断 和 小区不可用告警 的天数

    tmp = data[(data['告警名称'] == '网元连接中断') | (data['告警名称'] == '小区不可用告警')]
    tmp['start_time'] = tmp['告警开始时间'].apply(lambda x: str(x)[:10])
    tmp = tmp.drop_duplicates(subset=['ID', 'start_time'], keep='first')
    tmp = tmp.groupby(['ID']).agg({'start_time': 'nunique'}).reset_index().rename(columns={'start_time': 'label1_days'})
    res = res.merge(tmp, on='ID', how='left')

    # 最近一次发生 网元连接中断 和 小区不可用告警 的天数 距今的时间
    tmp = data[(data['告警名称'] == '网元连接中断') | (data['告警名称'] == '小区不可用告警')]
    tmp['告警开始时间'] = pd.to_datetime(tmp['告警开始时间'], format='%Y-%m-%d %H:%M:%S')
    tmp['end_time'] = pd.to_datetime(tmp['end_time'], format='%Y-%m-%d %H:%M:%S')
    tmp['near_label1_gaojing_gap'] = tmp.apply(lambda x: (x['end_time'] - x['告警开始时间']).days, axis=1)

    tmp2 = tmp.groupby(by=['ID', '告警名称']).agg({'near_label1_gaojing_gap': 'min'}).reset_index()
    tmp2 = pd.pivot_table(tmp2, index='ID', columns='告警名称', values='near_label1_gaojing_gap').reset_index().rename(
        columns={'网元连接中断': 'near_wangyuan_gap', '小区不可用告警': 'near_xiaoqu_gap'})
    res = res.merge(tmp2, on='ID', how='left')

    # 每个  网元连接中断 或 小区不可用告警 的时间间隔

    tmp3 = tmp
    rename = 'label1_gap'
    tmp3.sort_values(['ID', '告警开始时间'], inplace=True)
    tmp3['next_gaojing_time'] = tmp3.groupby(by=['ID'])['告警开始时间'].shift(-1)
    tmp3['gaojing_gaps'] = tmp3.apply(lambda x: (x['next_gaojing_time'] - x['告警开始时间']).seconds, axis=1)
    tmp3 = tmp3[['ID', '告警开始时间', 'next_gaojing_time', 'gaojing_gaps']]
    tmp3.dropna(subset=['gaojing_gaps'], inplace=True)

    tmp3 = tmp3.groupby(by=['ID'])['gaojing_gaps'].agg(['max', 'min', 'mean', 'std', 'skew']).reset_index().rename(
        columns={'max': rename + '_max', 'min': rename + '_min', 'mean': rename + '_mean', 'std': rename + '_std',
                 'skew': rename + '_skew'})
    res = res.merge(tmp3, on='ID', how='left')
    #     print(tmp3)

    # 统计每个告警类型，在7天内发生的次数

    tmp = data.groupby(by=['ID', '告警名称']).agg({'基站名称': 'count'}).reset_index()
    tmp = pd.pivot_table(tmp, index='ID', columns='告警名称', values='基站名称').reset_index()
    cols = {}
    i = 0
    for col in tmp.columns:
        if col not in ['ID', 'end_time', 'label']:
            cols[col] = i
            i += 1
    tmp.rename(columns=cols, inplace=True)
    res = res.merge(tmp, on='ID', how='left')
    res['0/sum7'] = res[0] / res['sum_guzhang_7']
    res['1/sum7'] = res[1] / res['sum_guzhang_7']
    res['3/sum7'] = res[3] / res['sum_guzhang_7']

    return res


def search_threthold(true, pred):
    score = 0
    bestThrethold = 0
    for i in np.arange(0, 1, 0.01):
        if f1_score(true, np.where(pred > i, 1, 0)) > score:
            score = f1_score(true, np.where(pred > i, 1, 0))
            bestThrethold = i
        else:
            pass
    return bestThrethold


def train_lgb_model(train_, valid_, valid_2, id_name, label_name, categorical_feature=None, seed=1024, is_shuffle=True):
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    train_['res'] = 0
    pred = [col for col in train_.columns if col not in [id_name, label_name, 'res']]
    print('特征数量为：', len(pred))
    sub_preds = np.zeros((valid_.shape[0], folds.n_splits))
    sub_preds2 = np.zeros((valid_2.shape[0], folds.n_splits))
    params = {
        'learning_rate': 0.01,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': ['binary_logloss', 'auc'],
        'num_leaves': 32,
        'feature_fraction': 0.7,
        'bagging_fraction': 0.7,
        'bagging_freq': 5,
        'seed': 1,
        # 'device': 'gpu',
        'bagging_seed': 1,
        'feature_fraction_seed': 7,
        'min_data_in_leaf': 28,
        'nthread': -1,
        'verbose': -1,
    }
    fea_impor = pd.DataFrame()
    fea_impor['column'] = train_[pred].columns
    fea_impor['importance'] = 0

    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_, train_[label_name]), start=1):
        print(f'the {n_fold} training start ...')

        train_x, train_y = train_[pred].iloc[train_idx], train_[label_name].iloc[train_idx]
        valid_x, valid_y = train_[pred].iloc[valid_idx], train_[label_name].iloc[valid_idx]

        dtrain = lgb.Dataset(train_x, label=train_y)
        dvalid = lgb.Dataset(valid_x, label=valid_y)

        clf = lgb.train(
            params=params,
            train_set=dtrain,
            num_boost_round=1000,
            valid_sets=[dtrain, dvalid],
            early_stopping_rounds=100,
            # feval=fscore,
            verbose_eval=100,
        )
        fea_impor['tmp'] = clf.feature_importance()
        fea_impor['importance'] = fea_impor['importance'] + fea_impor['tmp']

        sub_preds[:, n_fold - 1] = clf.predict(valid_[pred], num_iteration=clf.best_iteration)
        sub_preds2[:, n_fold - 1] = clf.predict(valid_2[pred], num_iteration=clf.best_iteration)
        train_pred = clf.predict(valid_x, num_iteration=clf.best_iteration)
        tmp_score = roc_auc_score(valid_y, train_pred)
        train_['res'].iloc[valid_idx] = train_['res'].iloc[valid_idx] + train_pred
        print(f'Orange roc_auc_score score: {tmp_score}')

    tmp_score = roc_auc_score(train_[label_name], train_['res'])
    print(f'five flod roc_auc_score score: {tmp_score}')
    train_.sort_values(by=['res'], ascending=False, inplace=True)

    # 按照0.5划分
    th = search_threthold(train_[label_name], train_['res'])
    train_['res'] = train_['res'].apply(lambda x: 1 if x > th else 0)

    tmp_f1 = f1_score(train_[label_name], train_['res'])

    print(f'five flod tmp_f1 score: {th, tmp_f1}')

    valid_[label_name] = np.mean(sub_preds, axis=1)
    valid_2[label_name] = np.mean(sub_preds2, axis=1)

    valid_['基站名称'] = valid_[id_name]
    valid_2['基站名称'] = valid_2[id_name]

    valid_['未来24小时发生退服类告警的概率'] = valid_[label_name]
    valid_2['未来24小时发生退服类告警的概率'] = valid_2[label_name]

    return th, valid_[['基站名称', '未来24小时发生退服类告警的概率']], \
           valid_2[['基站名称', '未来24小时发生退服类告警的概率']]



In [3]:
# 生成训练集
def gener_train_data(all_data, times):
    """
    start_i: 从第几天开始抽取
    times： 抽取多少次
    """
    res_data = pd.DataFrame()
    all_data['end_time'] = pd.to_datetime(all_data['end_time'], format='%Y-%m-%d %H:%M:%S')
    for i in tqdm(range(times)):
        # print(i)
        label_data = all_data[all_data['time_gap'] == i]
        # 生成label
        label_data['label'] = label_data['告警名称'].apply(
            lambda x: 1 if x.strip() == '网元连接中断' or x.strip() == '小区不可用告警' else 0)
        
        label_data = label_data.groupby('基站名称')['label'].agg('sum').reset_index()
        
        label_data['label'] = label_data['label'].apply(lambda x: 1 if x > 0 else 0)
        print(label_data)
        
        # 取前7天的训练数据
        tmp_data = all_data[(all_data['time_gap'] > i) & (all_data['time_gap'] <= i + 7)]
        # 处理时间 保持同一个窗口内，大小为7i
        tmp_data['time_gap'] = tmp_data['time_gap'] - i
        tmp_data = tmp_data.merge(label_data, on='基站名称', how='left')

        tmp_data['ID'] = tmp_data['基站名称'] + '_' + str(i)
        tmp_data['end_time'] = tmp_data['end_time'] - datetime.timedelta(days=i)

        res_data = res_data.append(tmp_data)
    return res_data


In [83]:
train_path = 'data'
test_0322_path = 'data'
test_0330_path = 'data'

all_test_data = pd.DataFrame()
for now_csv in tqdm(os.listdir(test_0330_path)):
    data = pd.read_csv(os.path.join(test_0330_path, now_csv))
    data['end_time'] = '2020-03-31'
    data['label'] = -1
    data['ID'] = data['基站名称']
    all_test_data = all_test_data.append(data)
all_test_data['end_time'] = pd.to_datetime(all_test_data['end_time'], format='%Y-%m-%d')
all_test_data['告警开始时间'] = pd.to_datetime(all_test_data['告警开始时间'], format='%Y-%m-%d %H:%M:%S')

all_test_data['time_gap'] = all_test_data.apply(lambda x: (x['end_time'] - x['告警开始时间']).days, axis=1)
# print(all_test_data['time_gap'])
all_test_data['time_gap'] += 1
# print(all_test_data['time_gap'])

save_path = './all_train_data.csv'
# if not os.path.exists(save_path):
#     all_train_data = get_all_train_data(train_path)
#     all_train_data.to_csv(save_path, index=False)
all_train_data = pd.read_csv('all_train_data.csv')
# offline_train_data = get_train_data(all_train_data, times=30)  # 后面没用到过？
online_train_data = get_train_data(all_train_data, times=3)
# all_data = online_train_data.append(all_test_data)  # 把测试集加入训练集

  0%|                                                 | 0/3 [00:00<?, ?it/s]d:\shayxu\pycharmprojects\littleprograms\venv\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
d:\shayxu\pycharmprojects\littleprograms\venv\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100%|█████████████████████████████████████████| 3/3 [00:00<00:00, 11.15it/s]

   基站名称  label
0  fake      1
   基站名称  label
0  fake      1
   基站名称  label
0  fake      0


In [84]:
online_train_data

,告警开始时间,基站名称,告警名称,end_time,time_gap,label,ID
0,2020-03-03 10:46:16,ACZDoAAEEAAAI1wABv,X2接口故障告警,2020-03-10,6,NaN,ACZDoAAEEAAAI1wABv_0
1,2020-03-03 11:26:25,ACZDoAAEEAAAI1wABv,BBU IR接口异常告警,2020-03-10,6,NaN,ACZDoAAEEAAAI1wABv_0
2,2020-03-03 11:45:00,ACZDoAAEEAAAI1wABv,[省内]4G小区状态异常,2020-03-10,6,NaN,ACZDoAAEEAAAI1wABv_0
3,2020-03-03 11:45:00,ACZDoAAEEAAAI1wABv,[省内]4G小区状态异常,2020-03-10,6,NaN,ACZDoAAEEAAAI1wABv_0
4,2020-03-03 15:15:00,ACZDoAAEEAAAI1wABv,[省内]4G小区状态异常,2020-03-10,6,NaN,ACZDoAAEEAAAI1wABv_0
5,2020-03-03 18:45:00,ACZDoAAEEAAAI1wABv,[省内]4G小区状态异常,2020-03-10,6,NaN,ACZDoAAEEAAAI1wABv_0
6,2020-03-04 00:12:15,ACZDoAAEEAAAI1wABv,X2接口故障告警,2020-03-10,5,NaN,ACZDoAAEEAAAI1wABv_0
7,2020-03-04 08:45:00,ACZDoAAEEAAAI1wABv,[省内]4G小区状态异常,2020-03-10,5,NaN,ACZDoAAEEAAAI1wABv_0
8,2020-03-04 15:35:35,ACZDoAAEEAAAI1wABv,射频单元光模块/电接口不在位告警,2020-03-10,5,NaN,ACZDoAAEEAAAI1wABv_0
9,2020-03-06 14:12:47,ACZDoAAEEAAAI1wABv,X2接口故障告警,2020-03-10,3,NaN,ACZDoAAEEAAAI1wABv_0


In [81]:
all_data = online_train_data
res_data = pd.DataFrame()
all_data['end_time'] = pd.to_datetime(all_data['end_time'], format='%Y-%m-%d %H:%M:%S')
i=1
label_data=all_data
label_data['label'] = label_data['告警名称'].apply(lambda x: 1 if x.strip() == '网元连接中断' or x.strip() == '小区不可用告警' else 0)
label_data = label_data.groupby('基站名称')['label'].agg('sum').reset_index()
label_data['label'] = label_data['label'].apply(lambda x: 1 if x > 0 else 0)

# # 取前7天的训练数据
tmp_data = all_data[(all_data['time_gap'] > i) & (all_data['time_gap'] <= i + 7)]
# # 处理时间 保持同一个窗口内，大小为7
tmp_data['time_gap'] = tmp_data['time_gap'] - i
tmp_data = tmp_data.merge(label_data, on='基站名称', how='left')

tmp_data['ID'] = tmp_data['基站名称'] + '_' + str(i)
print(tmp)
tmp_data['end_time'] = tmp_data['end_time'] - datetime.timedelta(days=i)

res_data = res_data.append(tmp_data)

                     ID                                               告警名称
0  ACZDoAAEEAAAI1wABv_0  X2接口故障告警 BBU IR接口异常告警 [省内]4G小区状态异常 [省内]4G小区状态异...
1  ACZDoAAEEAAAI1wABv_1  X2接口故障告警 BBU IR接口异常告警 [省内]4G小区状态异常 [省内]4G小区状态异...
2  ACZDoAAEEAAAI1wABv_2  X2接口故障告警 BBU IR接口异常告警 [省内]4G小区状态异常 [省内]4G小区状态异...
3  ACZDoAAEEAAAI1wABx_0  [省内]4G小区状态异常 [省内]4G小区状态异常 [省内]4G小区状态异常 [省内]4G小...
4  ACZDoAAEEAAAI1wABx_1  [省内]4G小区状态异常 [省内]4G小区状态异常 [省内]4G小区状态异常 [省内]4G小...
5  ACZDoAAEEAAAI1wABx_2  [省内]4G小区状态异常 [省内]4G小区状态异常 [省内]4G小区状态异常 [省内]4G小...
6                fake_0  X2接口故障告警 BBU IR接口异常告警 [省内]4G小区状态异常 [省内]4G小区状态异...
7                fake_1  X2接口故障告警 BBU IR接口异常告警 [省内]4G小区状态异常 [省内]4G小区状态异...
8                fake_2  X2接口故障告警 BBU IR接口异常告警 [省内]4G小区状态异常 [省内]4G小区状态异...


d:\shayxu\pycharmprojects\littleprograms\venv\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [82]:
res_data

,告警开始时间,基站名称,告警名称,end_time,time_gap,label_x,ID,label_y
0,2020-03-03 10:46:16,ACZDoAAEEAAAI1wABv,X2接口故障告警,2020-03-09,5,0,ACZDoAAEEAAAI1wABv_1,0
1,2020-03-03 11:26:25,ACZDoAAEEAAAI1wABv,BBU IR接口异常告警,2020-03-09,5,0,ACZDoAAEEAAAI1wABv_1,0
2,2020-03-03 11:45:00,ACZDoAAEEAAAI1wABv,[省内]4G小区状态异常,2020-03-09,5,0,ACZDoAAEEAAAI1wABv_1,0
3,2020-03-03 11:45:00,ACZDoAAEEAAAI1wABv,[省内]4G小区状态异常,2020-03-09,5,0,ACZDoAAEEAAAI1wABv_1,0
4,2020-03-03 15:15:00,ACZDoAAEEAAAI1wABv,[省内]4G小区状态异常,2020-03-09,5,0,ACZDoAAEEAAAI1wABv_1,0
5,2020-03-03 18:45:00,ACZDoAAEEAAAI1wABv,[省内]4G小区状态异常,2020-03-09,5,0,ACZDoAAEEAAAI1wABv_1,0
6,2020-03-04 00:12:15,ACZDoAAEEAAAI1wABv,X2接口故障告警,2020-03-09,4,0,ACZDoAAEEAAAI1wABv_1,0
7,2020-03-04 08:45:00,ACZDoAAEEAAAI1wABv,[省内]4G小区状态异常,2020-03-09,4,0,ACZDoAAEEAAAI1wABv_1,0
8,2020-03-04 15:35:35,ACZDoAAEEAAAI1wABv,射频单元光模块/电接口不在位告警,2020-03-09,4,0,ACZDoAAEEAAAI1wABv_1,0
9,2020-03-06 14:12:47,ACZDoAAEEAAAI1wABv,X2接口故障告警,2020-03-09,2,0,ACZDoAAEEAAAI1wABv_1,0


In [75]:
all_train_data = pd.read_csv('all_train_data.csv')

In [70]:
type(all_train_data['告警开始时间'][0])

str